<a href="https://colab.research.google.com/github/bhartiansh/pruning_comparison_research/blob/main/TinyImageNET_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://raw.githubusercontent.com/yandexdataschool/Practical_DL/spring2019/week03_convnets/tiny_img.py -O tiny_img.py

--2025-04-12 10:08:37--  https://raw.githubusercontent.com/yandexdataschool/Practical_DL/spring2019/week03_convnets/tiny_img.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3378 (3.3K) [text/plain]
Saving to: ‘tiny_img.py’

tiny_img.py         100%[===================>]   3.30K  --.-KB/s    in 0s      

2025-04-12 10:08:38 (58.3 MB/s) - ‘tiny_img.py’ saved [3378/3378]



In [13]:
import os
import zipfile
import shutil
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [20]:
!git clone https://github.com/bhartiansh/pruning_comparison_research
%cd pruning_comparison_research

Cloning into 'pruning_comparison_research'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.
/content/pruning_comparison_research/pruning_comparison_research/pruning_comparison_research


In [7]:
#  Extract Tiny ImageNet if not already extracted
zip_path = "/content/drive/MyDrive/tiny-imagenet-200.zip"  # update path if needed
extract_path = "/content/tiny-imagenet-200"

In [8]:
if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("/content/")

In [9]:
# Define paths
train_dir = os.path.join(extract_path, 'train')
val_dir = os.path.join(extract_path, 'val/images')

In [11]:
val_dir = "/content/tiny-imagenet-200/val"
val_images_dir = os.path.join(val_dir, 'images')
annotations_file = os.path.join(val_dir, 'val_annotations.txt')

# 📄 Read val_annotations.txt
df = pd.read_csv(annotations_file, sep="\t", header=None)
df.columns = ['file', 'class', 'x1', 'y1', 'x2', 'y2']

# 🗂️ Create folders and move images
for index, row in df.iterrows():
    class_dir = os.path.join(val_dir, 'images_by_class', row['class'])
    os.makedirs(class_dir, exist_ok=True)

    src = os.path.join(val_images_dir, row['file'])
    dst = os.path.join(class_dir, row['file'])
    if os.path.exists(src):
        shutil.move(src, dst)

# ✅ Update val_dir path to point to newly structured folder
val_dir = os.path.join(val_dir, 'images_by_class')

In [12]:
# Data Generators (64x64 is MobileNetV2’s minimum)
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=64,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(64, 64),
    batch_size=64,
    class_mode='categorical'
)

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.
